In [14]:
# EJECUTA ESTO PRIMERO en tu notebook
%load_ext kedro.ipython

# Esto automáticamente configurará todo y creará:
# - context, session, catalog, pipelines
print("✅ Kedro extension loaded!")

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython
✅ Kedro extension loaded!


In [15]:
print("Datasets disponibles en el catálogo:")
catalog.keys()



Datasets disponibles en el catálogo:



[
    'animal_charity_donation_records',
    'aac_intakes_outcomes',
    'cleaned_dog_breed',
    'cleaned_charity_donations_notebook',
    'parameters',
    'params:data_engineering',
    'params:data_engineering.imputation_strategy',
    'params:data_engineering.imputation_strategy.numerical',
    'params:data_engineering.imputation_strategy.categorical',
    'params:data_engineering.scaling',
    'params:data_engineering.scaling.standard_scaler',
    'params:data_engineering.scaling.robust_scaler',
    'params:data_engineering.outlier_limits',
    'params:data_engineering.outlier_limits.lower_percentile',
    'params:data_engineering.outlier_limits.upper_percentile',
    'params:data_engineering.outlier_limits.iqr_multiplier',
    'params:data_engineering.encode_columns',
    'synthetic_dog_breed_health_data'
]

In [16]:
df_dog_breed = catalog.load("synthetic_dog_breed_health_data")




print(f" df_dog_breed: {df_dog_breed.shape}")


[09/14/25 02:17:33] INFO     Loading data from synthetic_dog_breed_health_data (CSVDataset)... ]8;id=338809;file://C:\Users\alvaro\Documents\GitHub\MachineLearning\Proyecto_ML_1\.venv\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=362425;file://C:\Users\alvaro\Documents\GitHub\MachineLearning\Proyecto_ML_1\.venv\lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

 df_dog_breed: (10000, 21)


In [17]:
# NOTEBOOK 03: data_preparation.ipynb
# ==================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer

# Dataset ya cargado: df_donations
print(f"Dataset original: {df_dog_breed.shape}")
print(f"Valores missing iniciales: {df_dog_breed.isnull().sum().sum()}")

# 1. IMPUTACIÓN DE VALORES FALTANTES
print("\n1. IMPUTACIÓN DE VALORES FALTANTES")
print("=" * 40)

# Estrategias basadas en el análisis previo
imputation_strategy = {
    # Variables categóricas con >20% missing
    'Spay/Neuter Status': 'most_frequent',
    'Daily Activity Level': 'most_frequent', 
    'Owner Activity Level': 'most_frequent',
    
    # Variables categóricas con 2-3% missing
    'Breed': 'most_frequent',
    'Breed Size': 'most_frequent',
    'Sex': 'most_frequent',
    'Diet': 'most_frequent',
    'Food Brand': 'most_frequent',
    'Other Pets in Household': 'most_frequent',
    'Medications': 'most_frequent',
    'Seizures': 'most_frequent',
    'Synthetic': 'most_frequent',
    'Healthy': 'most_frequent',  # Variable objetivo importante
    
    # Variables numéricas
    'Age': 'median',
    'Weight (lbs)': 'median',
    'Daily Walk Distance (miles)': 'median',
    'Hours of Sleep': 'median', 
    'Play Time (hrs)': 'median',
    'Annual Vet Visits': 'median',
    'Average Temperature (F)': 'median'
}

# Aplicar imputación CORRECTAMENTE
df_clean = df_dog_breed.copy()
for column, strategy in imputation_strategy.items():
    if column in df_clean.columns:
        if strategy == 'most_frequent':
            imputer = SimpleImputer(strategy='most_frequent')
        else:
            imputer = SimpleImputer(strategy=strategy)
        
        # CORRECCIÓN: Convertir a 1D array con .ravel()
        imputed_data = imputer.fit_transform(df_clean[[column]]).ravel()
        df_clean[column] = imputed_data

print("✅ Imputación completada")
print(f"Valores missing restantes: {df_clean.isnull().sum().sum()}")

Dataset original: (10000, 21)
Valores missing iniciales: 13124

1. IMPUTACIÓN DE VALORES FALTANTES
✅ Imputación completada
Valores missing restantes: 0


In [18]:
# 2. MANEJO DE OUTLIERS
print("\n2. MANEJO DE OUTLIERS")
print("=" * 40)

# Para variables numéricas clave - winsorization suave
numeric_vars = ['Weight (lbs)', 'Daily Walk Distance (miles)', 'Play Time (hrs)', 'Annual Vet Visits']

for var in numeric_vars:
    Q1 = df_clean[var].quantile(0.05)  # Usamos percentiles 5 y 95 para winsorization
    Q3 = df_clean[var].quantile(0.95)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Reemplazar outliers con los límites
    df_clean[var] = np.where(df_clean[var] < lower_bound, lower_bound, df_clean[var])
    df_clean[var] = np.where(df_clean[var] > upper_bound, upper_bound, df_clean[var])
    
    print(f"{var}: outliers tratados con winsorization")

print("✅ Outliers manejados")


2. MANEJO DE OUTLIERS
Weight (lbs): outliers tratados con winsorization
Daily Walk Distance (miles): outliers tratados con winsorization
Play Time (hrs): outliers tratados con winsorization
Annual Vet Visits: outliers tratados con winsorization
✅ Outliers manejados


In [19]:
# 3. ESCALADO DE VARIABLES NUMÉRICAS
print("\n3. ESCALADO DE VARIABLES")
print("=" * 40)

# Separar variables para diferentes escalados
scaler_standard = StandardScaler()
scaler_robust = RobustScaler()

# Variables para StandardScaler (distribución normal)
standard_vars = ['Weight (lbs)']

# Variables para RobustScaler (con outliers)
robust_vars = ['Annual Vet Visits']

# Aplicar escalado
df_clean[standard_vars] = scaler_standard.fit_transform(df_clean[standard_vars])
df_clean[robust_vars] = scaler_robust.fit_transform(df_clean[robust_vars])

print("✅ Variables escaladas:")
print(f"   - StandardScaler: {standard_vars}")
print(f"   - RobustScaler: {robust_vars}")


3. ESCALADO DE VARIABLES
✅ Variables escaladas:
   - StandardScaler: ['Weight (lbs)']
   - RobustScaler: ['Annual Vet Visits']


In [20]:
# 4. CODIFICACIÓN DE VARIABLES CATEGÓRICAS
print("\n4. CODIFICACIÓN DE VARIABLES CATEGÓRICAS")
print("=" * 50)

# Identificar variables categóricas para encoding
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()
print(f"Variables categóricas a codificar: {categorical_cols}")

# Aplicar One-Hot Encoding a variables categóricas (excepto la target)
categorical_cols_to_encode = [col for col in categorical_cols if col != 'Healthy']

df_encoded = pd.get_dummies(df_clean, columns=categorical_cols_to_encode, prefix_sep='_', drop_first=True)

print(f"✅ Dataset después de encoding: {df_encoded.shape}")
print(f"Nuevas columnas: {df_encoded.columns.tolist()}")


4. CODIFICACIÓN DE VARIABLES CATEGÓRICAS
Variables categóricas a codificar: ['Breed', 'Breed Size', 'Sex', 'Spay/Neuter Status', 'Daily Activity Level', 'Diet', 'Food Brand', 'Other Pets in Household', 'Medications', 'Seizures', 'Owner Activity Level', 'Synthetic', 'Healthy']
✅ Dataset después de encoding: (10000, 47)
Nuevas columnas: ['ID', 'Age', 'Weight (lbs)', 'Daily Walk Distance (miles)', 'Hours of Sleep', 'Play Time (hrs)', 'Annual Vet Visits', 'Average Temperature (F)', 'Healthy', 'Breed_Beagle', 'Breed_Boxer', 'Breed_Bulldog', 'Breed_Chihuahua', 'Breed_Dachshund', 'Breed_Doberman', 'Breed_German Shepherd', 'Breed_Golden Retriever', 'Breed_Great Dane', 'Breed_Labrador Retriever', 'Breed_Poodle', 'Breed_Rottweiler', 'Breed_Siberian Husky', 'Breed_Yorkshire Terrier', 'Breed Size_Medium', 'Breed Size_Small', 'Sex_Male', 'Spay/Neuter Status_Spayed', 'Daily Activity Level_Low', 'Daily Activity Level_Moderate', 'Daily Activity Level_Very Active', 'Diet_Home cooked', 'Diet_Special di

In [21]:
# CORREGIR VALORES NEGATIVOS EN VISITAS VETERINARIAS
print("\n CORRECCIÓN DE VALORES NEGATIVOS")
print("=" * 45)

# Reemplazar valores negativos por 0 (ninguna visita)
valores_negativos = (df_encoded['Annual Vet Visits'] < 0).sum()
print(f"Valores negativos encontrados: {valores_negativos}")

df_encoded['Annual Vet Visits'] = df_encoded['Annual Vet Visits'].clip(lower=0)
print(f"Valores negativos después de corrección: {(df_encoded['Annual Vet Visits'] < 0).sum()}")

# Ver distribución corregida
print("\n DISTRIBUCIÓN CORREGIDA DE VISITAS VETERINARIAS:")
print(df_encoded['Annual Vet Visits'].value_counts().sort_index())


 CORRECCIÓN DE VALORES NEGATIVOS
Valores negativos encontrados: 2161
Valores negativos después de corrección: 0

 DISTRIBUCIÓN CORREGIDA DE VISITAS VETERINARIAS:
Annual Vet Visits
0.0    5705
1.0    2431
2.0    1260
3.0     604
Name: count, dtype: int64


In [22]:
# VERIFICAR OTROS VALORES NEGATIVOS
print("\n🔍 REVISIÓN DE OTRAS VARIABLES NUMÉRICAS:")
numeric_cols = ['Weight (lbs)', 'Daily Walk Distance (miles)', 
                'Hours of Sleep', 'Play Time (hrs)', 'Annual Vet Visits']

for col in numeric_cols:
    if col in df_encoded.columns:
        negativos = (df_encoded[col] < 0).sum()
        if negativos > 0:
            print(f"{col}: {negativos} valores negativos")
        else:
            print(f"✅ {col}: Sin valores negativos")


🔍 REVISIÓN DE OTRAS VARIABLES NUMÉRICAS:
Weight (lbs): 5217 valores negativos
✅ Daily Walk Distance (miles): Sin valores negativos
✅ Hours of Sleep: Sin valores negativos
✅ Play Time (hrs): Sin valores negativos
✅ Annual Vet Visits: Sin valores negativos


In [23]:
# CORREGIR VALORES NEGATIVOS EN PESOS
print("\n🔧 CORRECCIÓN DE VALORES NEGATIVOS EN PESOS")
print("=" * 55)

# Ver distribución actual problemática
print("Distribución actual de Weight (lbs):")
print(df_encoded['Weight (lbs)'].describe())

# Los pesos NUNCA deberían ser negativos - aplicar límite inferior 0
df_encoded['Weight (lbs)'] = df_encoded['Weight (lbs)'].clip(lower=0)

print("\nDistribución corregida:")
print(df_encoded['Weight (lbs)'].describe())
print(f"Valores negativos después de corrección: {(df_encoded['Weight (lbs)'] < 0).sum()}")


🔧 CORRECCIÓN DE VALORES NEGATIVOS EN PESOS
Distribución actual de Weight (lbs):
count    1.000000e+04
mean     2.188472e-16
std      1.000050e+00
min     -2.738021e+00
25%     -6.945488e-01
50%     -1.339155e-02
75%      6.677657e-01
max      4.005436e+00
Name: Weight (lbs), dtype: float64

Distribución corregida:
count    10000.000000
mean         0.394824
std          0.588358
min          0.000000
25%          0.000000
50%          0.000000
75%          0.667766
max          4.005436
Name: Weight (lbs), dtype: float64
Valores negativos después de corrección: 0


In [24]:
# REVISIÓN FINAL DE TODAS LAS VARIABLES
print("\nREVISIÓN FINAL - VALORES NEGATIVOS:")
numeric_cols = ['Weight (lbs)', 'Daily Walk Distance (miles)', 
                'Hours of Sleep', 'Play Time (hrs)', 'Annual Vet Visits']

for col in numeric_cols:
    if col in df_encoded.columns:
        negativos = (df_encoded[col] < 0).sum()
        min_val = df_encoded[col].min()
        max_val = df_encoded[col].max()
        
        if negativos > 0:
            print(f" {col}: {negativos} negativos (min: {min_val:.2f}, max: {max_val:.2f})")
        else:
            print(f"✅ {col}: Sin negativos (min: {min_val:.2f}, max: {max_val:.2f})")


REVISIÓN FINAL - VALORES NEGATIVOS:
✅ Weight (lbs): Sin negativos (min: 0.00, max: 4.01)
✅ Daily Walk Distance (miles): Sin negativos (min: 0.00, max: 8.00)
✅ Hours of Sleep: Sin negativos (min: 8.00, max: 14.00)
✅ Play Time (hrs): Sin negativos (min: 0.00, max: 4.00)
✅ Annual Vet Visits: Sin negativos (min: 0.00, max: 3.00)


In [29]:
# GUARDAR DATASET CORRECTAMENTE
import os
from pathlib import Path

# 1. Primero crear la carpeta si no existe
os.makedirs('data/02_intermediate', exist_ok=True)

# 2. Guardar el DataFrame como CSV
output_path = 'data/02_intermediate/cleaned_dog_breed_notebook.csv'
df_encoded.to_csv(output_path, index=False)  # ← ¡ESTA LÍNEA FALTA!

print(f"💾 Dataset guardado como: {output_path}")

# 3. Verificar que se creó
file_path = Path(output_path)
if file_path.exists():
    print(f"✅ Archivo verificado: {file_path} ({file_path.stat().st_size} bytes)")
else:
    print("❌ Error: El archivo no se creó")

💾 Dataset guardado como: data/02_intermediate/cleaned_dog_breed_notebook.csv
✅ Archivo verificado: data\02_intermediate\cleaned_dog_breed_notebook.csv (2683214 bytes)


In [26]:

print("\n6. VERIFICACIÓN FINAL")
print("=" * 40)

# Mostrar información del dataset limpio
print("📝 INFORMACIÓN DEL DATASET LIMPIO:")
print(df_encoded.info())

# Mostrar primeras filas
print("\n👀 PRIMERAS 5 FILAS:")
display(df_encoded.head())


6. VERIFICACIÓN FINAL
📝 INFORMACIÓN DEL DATASET LIMPIO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 47 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                10000 non-null  int64  
 1   Age                               10000 non-null  float64
 2   Weight (lbs)                      10000 non-null  float64
 3   Daily Walk Distance (miles)       10000 non-null  float64
 4   Hours of Sleep                    10000 non-null  float64
 5   Play Time (hrs)                   10000 non-null  float64
 6   Annual Vet Visits                 10000 non-null  float64
 7   Average Temperature (F)           10000 non-null  float64
 8   Healthy                           10000 non-null  object 
 9   Breed_Beagle                      10000 non-null  bool   
 10  Breed_Boxer                       10000 non-null  bool   
 11  Breed_Bulld

,ID,Age,Weight (lbs),Daily Walk Distance (miles),Hours of Sleep,Play Time (hrs),Annual Vet Visits,Average Temperature (F),Healthy,Breed_Beagle,...,Food Brand_Purina,Food Brand_Royal Canin,Food Brand_Special,Food Brand_Wellness,Other Pets in Household_Yes,Medications_Yes,Seizures_Yes,Owner Activity Level_Low,Owner Activity Level_Moderate,Owner Activity Level_Very Active
0,1,3.0,0.667766,5.0,12.0,1.0,0.0,30.0,Yes,False,...,False,False,False,True,False,True,False,False,False,False
1,2,4.0,0.463419,2.0,11.0,3.0,0.0,47.0,Yes,False,...,False,False,True,False,True,False,False,False,False,True
2,3,12.0,1.144576,1.0,10.0,1.0,0.0,92.0,Yes,False,...,False,False,True,False,True,False,False,True,False,False
3,4,13.0,0.000000,4.0,12.0,2.0,0.0,75.0,Yes,False,...,False,False,True,False,False,False,False,False,False,True
4,5,13.0,0.000000,3.0,9.0,1.0,0.0,64.0,No,False,...,True,False,False,False,True,True,False,False,False,True
